In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`

import $ivy.$

In [2]:
// neo4j-spark-connector
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://repos.spark-packages.org/"))
interp.load.ivy(("neo4j" % "neo4j-spark-connector" % "5.3.3-s_2.12"))

import coursierapi._

## Spark Session с подключением к Neo4j

In [3]:
import org.apache.spark.sql._

import org.apache.spark.sql._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("SparkNeo4j")
                .config("spark.log.level", "WARN")
                .config("spark.jars.packages", "neo4j:neo4j-spark-connector:5.3.3-s_2.12")
                .config("neo4j.url", "neo4j://localhost:7687")
                .config("neo4j.authentication.type", "basic")
                .config("neo4j.authentication.basic.username", "neo4j")
                .config("neo4j.authentication.basic.password", "password")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/14 12:24:39 INFO SparkContext: Running Spark version 3.5.5
25/03/14 12:24:39 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/14 12:24:39 INFO SparkContext: Java version 11.0.26
25/03/14 12:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@702e8360
import spark.implicits._

## Read data from Neo4j into Spark

### Movies

In [5]:
val movies = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("labels", ":Movie")
                .load()

25/03/14 12:24:42 WARN SchemaService: Switching to query schema resolution


movies: DataFrame = [<id>: bigint, <labels>: array<string> ... 3 more fields]

In [6]:
movies.printSchema()

root
 |-- <id>: long (nullable = false)
 |-- <labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- released: long (nullable = true)



In [7]:
movies.show(10, false)

+----+--------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|<id>|<labels>|title                 |tagline                                                                                                                                                                      |released|
+----+--------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|0   |[Movie] |The Matrix            |Welcome to the Real World                                                                                                                                                    |1999    |
|10  |[Movie] |The Matrix Reloaded   |Free your mind                                                            

### Person

In [8]:
val person = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("labels", ":Person")
                .load()

25/03/14 12:24:47 WARN SchemaService: Switching to query schema resolution


person: DataFrame = [<id>: bigint, <labels>: array<string> ... 2 more fields]

In [9]:
person.printSchema()

root
 |-- <id>: long (nullable = false)
 |-- <labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- born: long (nullable = true)



In [10]:
person.show(10, false)

+----+--------+------------------+----+
|<id>|<labels>|name              |born|
+----+--------+------------------+----+
|1   |[Person]|Keanu Reeves      |1964|
|2   |[Person]|Carrie-Anne Moss  |1967|
|3   |[Person]|Laurence Fishburne|1961|
|4   |[Person]|Hugo Weaving      |1960|
|5   |[Person]|Lilly Wachowski   |1967|
|6   |[Person]|Lana Wachowski    |1965|
|7   |[Person]|Joel Silver       |1952|
|8   |[Person]|Emil Eifrem       |1978|
|27  |[Person]|Charlize Theron   |1975|
|28  |[Person]|Al Pacino         |1940|
+----+--------+------------------+----+
only showing top 10 rows



### ACTED_IN

In [11]:
val actedin = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("relationship", "ACTED_IN")
                .option("relationship.source.labels", ":Person")
                .option("relationship.target.labels", ":Movie")
                .load()

25/03/14 12:24:48 WARN SchemaService: Switching to query schema resolution
25/03/14 12:24:48 WARN SchemaService: Switching to query schema resolution
25/03/14 12:24:48 WARN SchemaService: Switching to query schema resolution


actedin: DataFrame = [<rel.id>: bigint, <rel.type>: string ... 10 more fields]

In [12]:
actedin.printSchema()

root
 |-- <rel.id>: long (nullable = false)
 |-- <rel.type>: string (nullable = false)
 |-- <source.id>: long (nullable = false)
 |-- <source.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source.name: string (nullable = true)
 |-- source.born: long (nullable = true)
 |-- <target.id>: long (nullable = false)
 |-- <target.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- target.title: string (nullable = true)
 |-- target.tagline: string (nullable = true)
 |-- target.released: long (nullable = true)
 |-- rel.roles: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
actedin.show(10, false)

+--------+----------+-----------+---------------+------------------+-----------+-----------+---------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------------------------+
|<rel.id>|<rel.type>|<source.id>|<source.labels>|source.name       |source.born|<target.id>|<target.labels>|target.title        |target.tagline                                                                                                                                                               |target.released|rel.roles                          |
+--------+----------+-----------+---------------+------------------+-----------+-----------+---------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------

### DataFrame with nodes as map

In [14]:
val actedinMap = spark.read
                    .format("org.neo4j.spark.DataSource")
                    .option("relationship.nodes.map", true)
                    .option("relationship", "ACTED_IN")
                    .option("relationship.source.labels", ":Person")
                    .option("relationship.target.labels", ":Movie")
                    .load()

25/03/14 12:24:50 WARN SchemaService: Switching to query schema resolution


actedinMap: DataFrame = [<rel.id>: bigint, <rel.type>: string ... 3 more fields]

In [15]:
actedinMap.printSchema()

root
 |-- <rel.id>: long (nullable = false)
 |-- <rel.type>: string (nullable = false)
 |-- <source>: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- <target>: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- rel.roles: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [16]:
actedinMap.show(10, false)

+--------+----------+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------+
|<rel.id>|<rel.type>|<source>                                                                       |<target>                                                                                                                                                                                                                                                                    |rel.roles                          |
+--------+----------+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------------

### Directed

In [17]:
val directed = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("relationship", "DIRECTED")
                .option("relationship.source.labels", ":Person")
                .option("relationship.target.labels", ":Movie")
                .load()

25/03/14 12:24:51 WARN SchemaService: Switching to query schema resolution
25/03/14 12:24:51 WARN SchemaService: Switching to query schema resolution
25/03/14 12:24:51 WARN SchemaService: Switching to query schema resolution


directed: DataFrame = [<rel.id>: bigint, <rel.type>: string ... 9 more fields]

In [18]:
directed.printSchema()

root
 |-- <rel.id>: long (nullable = false)
 |-- <rel.type>: string (nullable = false)
 |-- <source.id>: long (nullable = false)
 |-- <source.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source.name: string (nullable = true)
 |-- source.born: long (nullable = true)
 |-- <target.id>: long (nullable = false)
 |-- <target.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- target.title: string (nullable = true)
 |-- target.tagline: string (nullable = true)
 |-- target.released: long (nullable = true)



In [19]:
directed.show(10, false)

+--------+----------+-----------+---------------+---------------+-----------+-----------+---------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|<rel.id>|<rel.type>|<source.id>|<source.labels>|source.name    |source.born|<target.id>|<target.labels>|target.title          |target.tagline                                                                                                                                                               |target.released|
+--------+----------+-----------+---------------+---------------+-----------+-----------+---------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|5       |DIRECTED  |6          |[Person]  

### Read arbitrary data via Cypher query

In [20]:
val cypher = spark.read
                .format("org.neo4j.spark.DataSource")
                .option("query", """
                    // Extend Tom Hanks co-actors, to find co-co-actors who haven't worked with Tom Hanks
                    MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
                    (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
                    WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors)
                        AND tom <> cocoActors
                    RETURN cocoActors.name AS Recommended, count(*) AS Strength
                    ORDER BY Strength DESC
                    """)
                .load()

cypher: DataFrame = [Recommended: string, Strength: bigint]

In [21]:
cypher.printSchema()

root
 |-- Recommended: string (nullable = true)
 |-- Strength: long (nullable = true)



In [22]:
cypher.show(10, false)

+----------------+--------+
|Recommended     |Strength|
+----------------+--------+
|Zach Grenier    |10      |
|Tom Cruise      |10      |
|Keanu Reeves    |8       |
|Cuba Gooding Jr.|8       |
|Jack Nicholson  |6       |
|Kelly McGillis  |6       |
|Anthony Edwards |6       |
|Carrie Fisher   |6       |
|Bruno Kirby     |6       |
|Billy Crystal   |6       |
+----------------+--------+
only showing top 10 rows



### Return all the actors that have also directed a movie.

In [23]:
val actorsDirectors = spark.read
                        .format("org.neo4j.spark.DataSource")
                        .option("query", """
                                MATCH (p:Person)
                                MATCH (p)-[:ACTED_IN]->(m:Movie)
                                MATCH (p)-[:DIRECTED]->(m1:Movie)
                                RETURN p.name AS name, collect(m.title) AS acted_in, collect(m1.title) AS directed
                                """)
                        .load()

actorsDirectors: DataFrame = [name: string, acted_in: array<string> ... 1 more field]

In [24]:
actorsDirectors.printSchema()

root
 |-- name: string (nullable = true)
 |-- acted_in: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- directed: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [25]:
actorsDirectors.show(10, false)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|name          |acted_in                                                                     

## Write data from Spark to Neo4j

In [26]:
val products = spark.read
                .format("csv")
                .option("inferSchema", true)
                .load("desktop-csv-import/products.csv")
                .withColumnsRenamed(Map("_c0" -> "id", "_c1" -> "name", "_c2" -> "price"))

products: DataFrame = [id: int, name: string ... 1 more field]

In [27]:
products.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)



In [28]:
products.show(10, false)

+---+-------------------------------+-----+
|id |name                           |price|
+---+-------------------------------+-----+
|1  |Chai                           |18.0 |
|2  |Chang                          |19.0 |
|3  |Aniseed Syrup                  |10.0 |
|4  |Chef Anton's Cajun Seasoning   |22.0 |
|5  |Chef Anton's Gumbo Mix         |21.35|
|6  |Grandma's Boysenberry Spread   |25.0 |
|7  |Uncle Bob's Organic Dried Pears|30.0 |
|8  |Northwoods Cranberry Sauce     |40.0 |
|9  |Mishi Kobe Niku                |97.0 |
|10 |Ikura                          |31.0 |
+---+-------------------------------+-----+
only showing top 10 rows



In [29]:
products.count()

res29: Long = 77L

In [30]:
val orders = spark.read
                .format("csv")
                .option("inferSchema", true)
                .option("header", true)
                .load("desktop-csv-import/orders.csv")
                .selectExpr("orderID AS id", "CAST(orderDate AS TIMESTAMP) AS date", "shipCountry")

orders: DataFrame = [id: int, date: timestamp ... 1 more field]

In [31]:
orders.printSchema()

root
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- shipCountry: string (nullable = true)



In [32]:
orders.show(10, false)

+-----+-------------------+-----------+
|id   |date               |shipCountry|
+-----+-------------------+-----------+
|10248|1996-07-04 00:00:00|France     |
|10249|1996-07-05 00:00:00|Germany    |
|10250|1996-07-08 00:00:00|Brazil     |
|10251|1996-07-08 00:00:00|France     |
|10252|1996-07-09 00:00:00|Belgium    |
|10253|1996-07-10 00:00:00|Brazil     |
|10254|1996-07-11 00:00:00|Switzerland|
|10255|1996-07-12 00:00:00|Switzerland|
|10256|1996-07-15 00:00:00|Brazil     |
|10257|1996-07-16 00:00:00|Venezuela  |
+-----+-------------------+-----------+
only showing top 10 rows



In [33]:
orders.count()

res33: Long = 830L

### Write nodes via label option

In [34]:
products.write
        .format("org.neo4j.spark.DataSource")
        .mode("append")
        .option("labels", ":Product")
        .save()

In [35]:
orders.write
      .format("org.neo4j.spark.DataSource")
      .mode("overwrite")
      .option("labels", ":Order")
      .option("schema.optimization.type", "NODE_CONSTRAINTS")
// this is necessary in order to specify what is the constraint field
      .option("node.keys", "id")
      .save()

25/03/14 12:25:01 WARN Neo4jOptions: 
Option `schema.optimization.type` is deprecated and will be removed in future implementations,
please move to one of the following depending on your use case:
- `schema.optimization.node.keys`
- `schema.optimization.relationship.keys`

25/03/14 12:25:01 WARN Neo4jOptions: 
Option `schema.optimization.type` is deprecated and will be removed in future implementations,
please move to one of the following depending on your use case:
- `schema.optimization.node.keys`
- `schema.optimization.relationship.keys`



### Write relationships via relationship option

In [36]:
val orderDetails = spark.read
                    .format("csv")
                    .option("inferSchema", true)
                    .option("header", true)
                    .load("desktop-csv-import/order-details.csv")

orderDetails: DataFrame = [orderID: int, productID: int ... 1 more field]

In [37]:
orderDetails.printSchema()

root
 |-- orderID: integer (nullable = true)
 |-- productID: integer (nullable = true)
 |-- quantity: integer (nullable = true)



In [38]:
orderDetails.show(10, false)

+-------+---------+--------+
|orderID|productID|quantity|
+-------+---------+--------+
|10248  |11       |12      |
|10248  |42       |10      |
|10248  |72       |5       |
|10249  |14       |9       |
|10249  |51       |40      |
|10250  |41       |10      |
|10250  |51       |35      |
|10250  |65       |15      |
|10251  |22       |6       |
|10251  |57       |15      |
+-------+---------+--------+
only showing top 10 rows



In [39]:
orderDetails.count()

res39: Long = 2155L

In [40]:
orderDetails.write
            .format("org.neo4j.spark.DataSource")
            .mode("overwrite")
            .option("relationship", "CONTAINS")
            .option("relationship.save.strategy", "keys")
            .option("relationship.source.labels", ":Product")
            .option("relationship.source.save.mode", "Match")
            .option("relationship.source.node.keys", "productID:id")
            .option("relationship.target.labels", ":Order")
            .option("relationship.target.save.mode", "Match")
            .option("relationship.target.node.keys", "orderID:id")
            .option("relationship.properties", "quantity:quantityOrdered")
            .save()

### Write custom graphs via Cypher Query

In [41]:
val actorOrders = Seq(
  ("Cuba Gooding Jr.", 1, Array(11, 42, 72), Array(1, 2, 3), "2022-06-07 00:00:00"),
  ("Tom Hanks", 2, Array(24, 55, 75), Array(3, 2, 1), "2022-06-06 00:00:00")
)

actorOrders: Seq[(String, Int, Array[Int], Array[Int], String)] = List(
  (
    "Cuba Gooding Jr.",
    1,
    Array(11, 42, 72),
    Array(1, 2, 3),
    "2022-06-07 00:00:00"
  ),
  ("Tom Hanks", 2, Array(24, 55, 75), Array(3, 2, 1), "2022-06-06 00:00:00")
)

In [42]:
val actorOrdersDF = spark
                    .createDataFrame(actorOrders)
                    .withColumnsRenamed(
                        Map(
                            "_1" -> "actor_name",
                            "_2" -> "order_id",
                            "_3" -> "products",
                            "_4" -> "quantities",
                            "_5" -> "order_date"))

actorOrdersDF: DataFrame = [actor_name: string, order_id: int ... 3 more fields]

In [43]:
actorOrdersDF.printSchema()

root
 |-- actor_name: string (nullable = true)
 |-- order_id: integer (nullable = false)
 |-- products: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- quantities: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- order_date: string (nullable = true)



In [44]:
actorOrdersDF.show(10, false)

+----------------+--------+------------+----------+-------------------+
|actor_name      |order_id|products    |quantities|order_date         |
+----------------+--------+------------+----------+-------------------+
|Cuba Gooding Jr.|1       |[11, 42, 72]|[1, 2, 3] |2022-06-07 00:00:00|
|Tom Hanks       |2       |[24, 55, 75]|[3, 2, 1] |2022-06-06 00:00:00|
+----------------+--------+------------+----------+-------------------+



In [45]:
actorOrdersDF.write
             .format("org.neo4j.spark.DataSource")
             .mode("overwrite")
             .option("query", """
                     MATCH (person:Person {name: event.actor_name})
                     MERGE (order:Order {id: event.order_id, date: datetime(replace(event.order_date, ' ', 'T'))})
                     MERGE (person)-[:CREATED]->(order)
                     WITH event, order
                     UNWIND range(0, size(event.products) - 1) AS index
                     MATCH (product:Product {id: event.products[index]})
                     MERGE (product)-[:CONTAINS{quantityOrdered: event.quantities[index]}]->(order)
                    """)
             .save()